In [1]:
import numpy as np
import category_encoders as ce
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, LabelBinarizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import roc_auc_score
import pandas as pd
import joblib

In [2]:
bank_test = pd.read_csv("./data/interim/bank_teste.csv")

In [3]:
X_teste = bank_test.drop("y", axis=1)
y_teste = LabelBinarizer().fit_transform(bank_test[["y"]])

In [5]:
lr_clf = joblib.load("./models/ab.pkl.old")

In [6]:
pd.DataFrame(lr_clf.cv_results_).sort_values("rank_test_score")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__learning_rate,param_clf__n_estimators,param_transf_features__te_job__sigma,param_transf_features__te_job__smoothing,param_transf_features__te_job__transformer,params,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,49.754144,4.735261,0.532279,0.086087,1,300,0.950714,1.463988,target_encoder,"{'clf__learning_rate': 1, 'clf__n_estimators':...",...,0.732234,0.704825,0.712813,0.752890,0.699112,0.736773,0.710788,0.720825,0.016479,1
16,39.259836,3.408854,0.458065,0.026714,1,300,0.045227,0.650661,target_encoder,"{'clf__learning_rate': 1, 'clf__n_estimators':...",...,0.732234,0.704825,0.712813,0.752890,0.699112,0.736773,0.710788,0.720825,0.016479,1
18,19.146127,0.278298,0.175241,0.015423,0.01,50,0.802197,0.149101,one_hot_encoder,"{'clf__learning_rate': 0.01, 'clf__n_estimator...",...,0.720718,0.716272,0.699741,0.738511,0.709870,0.716859,0.710425,0.713259,0.011730,3
3,35.304442,7.719973,0.313924,0.074681,0.01,100,0.721999,1.877105,leave_one_out,"{'clf__learning_rate': 0.01, 'clf__n_estimator...",...,0.714909,0.720902,0.695679,0.734022,0.717709,0.713265,0.708394,0.711365,0.011628,4
10,41.098296,4.022847,0.317831,0.033575,0.01,100,0.097672,1.368466,one_hot_encoder,"{'clf__learning_rate': 0.01, 'clf__n_estimator...",...,0.718674,0.716627,0.692726,0.730885,0.715079,0.709999,0.702267,0.710947,0.011290,5
6,72.290546,4.079445,0.646462,0.079929,0.1,300,0.292145,0.732724,leave_one_out,"{'clf__learning_rate': 0.1, 'clf__n_estimators...",...,0.717923,0.703621,0.700584,0.729824,0.713356,0.714048,0.715290,0.710263,0.009487,6
9,66.800374,4.867241,0.694047,0.146566,0.1,300,0.942202,1.126576,leave_one_out,"{'clf__learning_rate': 0.1, 'clf__n_estimators...",...,0.717923,0.703621,0.700584,0.729824,0.713356,0.714048,0.715290,0.710263,0.009487,6
1,101.481904,10.185823,0.596955,0.140069,0.01,200,0.445833,0.19995,one_hot_encoder,"{'clf__learning_rate': 0.01, 'clf__n_estimator...",...,0.723404,0.715391,0.682938,0.726105,0.717113,0.704852,0.707789,0.708500,0.016838,8
5,120.757442,8.641481,0.850150,0.150983,0.01,300,0.023062,1.049549,one_hot_encoder,"{'clf__learning_rate': 0.01, 'clf__n_estimator...",...,0.723269,0.716971,0.682688,0.715019,0.716195,0.705396,0.715294,0.707047,0.017435,9
19,102.758527,12.026115,0.898371,0.339922,0.01,300,0.394882,0.586976,one_hot_encoder,"{'clf__learning_rate': 0.01, 'clf__n_estimator...",...,0.723269,0.716971,0.682688,0.715019,0.716195,0.705396,0.715294,0.707047,0.017435,9


In [7]:
bank_predictions = lr_clf.best_estimator_.predict(X_teste)

In [8]:
roc_auc_score(y_teste, bank_predictions)

0.6139887802411256

In [11]:
bank_train = pd.read_csv("../data/interim/bank_train.csv")

In [12]:
X = bank_train.copy().drop("y", axis=1)
y = bank_train.copy()[["y"]]

In [13]:
lb = LabelBinarizer()
y = lb.fit_transform(y)

In [120]:
class JobTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, transformer="one_hot_encoder", smoothing=1, sigma=0.05):
        self.transformer = transformer
        self.smoothing = smoothing
        self.sigma = sigma
        
    def fit(self, X, y=None):
        if self.transformer == "target_encoder":
            self.encoder = ce.TargetEncoder(smoothing=self.smoothing)
        elif self.transformer == "leave_one_out":
            self.encoder = ce.LeaveOneOutEncoder(random_state=42, sigma=self.sigma)
        elif self.transformer == "one_hot_encoder":
            self.encoder = ce.OneHotEncoder(use_cat_names=True)
        self.encoder.fit(X, y)
        return self
    
    def transform(self, X, y=None):
        if self.transformer == "one_hot_encoder":
            return self.encoder.transform(X)
        else:
            return self.encoder.transform(X, y)

In [121]:
je = JobTransformer(transformer="leave_one_out")
je.fit_transform(X["job"], y)

,job
0,0.130826
1,0.110395
2,0.070000
3,0.083660
4,0.130826
...,...
28826,0.242373
28827,0.112711
28828,0.130826
28829,0.112711


In [116]:
je = JobTransformer(transformer="one_hot_encoder")
je.fit_transform(X["job"], y)

AttributeError: 'JobTransformer' object has no attribute 'encoder'

In [40]:
le = ce.LeaveOneOutEncoder()
le.fit_transform(X[["job"]], y)

,job
0,0.130844
1,0.110419
2,0.070011
3,0.083689
4,0.130844
...,...
28826,0.242578
28827,0.112767
28828,0.130707
28829,0.112767


In [33]:
je.transform(X)

ValueError: Supervised encoders need a target for the fitting. The target cannot be None

In [15]:
y

array([[0],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]])

In [14]:
X

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,29,admin.,married,university.degree,no,no,no,cellular,dec,mon,77,3,999,1,failure,-3.0,92.713,-33.0,0.709,5023.5
1,29,technician,single,university.degree,no,no,no,telephone,may,fri,12,4,999,0,nonexistent,-1.8,92.893,-46.2,1.250,5099.1
2,45,blue-collar,married,basic.6y,unknown,yes,no,telephone,may,thu,277,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0
3,34,services,married,university.degree,no,no,no,cellular,may,thu,70,1,999,1,failure,-1.8,92.893,-46.2,1.327,5099.1
4,32,admin.,single,high.school,no,no,no,cellular,may,fri,1181,9,999,0,nonexistent,-1.8,92.893,-46.2,1.250,5099.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,58,retired,married,professional.course,unknown,no,no,telephone,may,tue,427,2,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
28827,37,management,married,university.degree,no,no,no,telephone,jun,thu,288,1,999,0,nonexistent,1.4,94.465,-41.8,4.961,5228.1
28828,35,admin.,married,high.school,no,yes,no,cellular,oct,thu,194,1,4,1,success,-3.4,92.431,-26.9,0.754,5017.5
28829,40,management,married,university.degree,no,yes,no,telephone,may,wed,295,2,999,0,nonexistent,1.1,93.994,-36.4,4.856,5191.0


In [13]:

ordinalencoder_mapping = [
    {"col": "education",
     "mapping": {
         "unknown": 0,
         "illiterate": 0,
         "basic.4y": 1,
         "basic.6y": 1,
         "basic.9y": 1,
         "high.school": 2,
         "professional.course": 3,
         "university.degree": 4
     }},
    {"col": "month",
     "mapping": {
         "jan": 1,
         "feb": 2,
         "mar": 3,
         "apr": 4, 
         "may": 5, 
         "jun": 6, 
         "jul": 7, 
         "aug": 8, 
         "sep": 9, 
         "oct": 10, 
         "nov": 11, 
         "dec": 12
     }},
    {"col": "day_of_week",
     "mapping": {
         "mon": 1, 
         "tue": 2, 
         "wed": 3, 
         "thu": 4, 
         "fri": 5 
     }}
]

pipe_pca = Pipeline([
    ("std", StandardScaler()),
    ("pca", PCA(n_components=1))
])




pipe_pca = Pipeline([
    ("std", StandardScaler()),
    ("pca", PCA(n_components=1))
])



col_transf = ColumnTransformer([
    ("bin_age", 
     FunctionTransformer(
         func=lambda x: np.select([(x < 30), (x >= 30) & (x < 60), (x >= 60)],
                                  [1, 2, 3])),
     ["age"]
    ),
    ("te_job", ce.TargetEncoder(), ["job"]),
    ("oh_marital", OneHotEncoder(drop=["unknown"]), ["marital"]),
    ("oe_vars", 
     ce.OrdinalEncoder(mapping=ordinalencoder_mapping), 
     ["education", "month", "day_of_week"]),
    ("oh_default", OneHotEncoder(drop=["yes"]), ["default"]),
    ("oh_housing_loan_contact", OneHotEncoder(), ["housing", "loan", "contact", "poutcome"]),
    ("bin_campaign",
     FunctionTransformer(
         func=lambda x: np.select([(x == 1), (x == 2), (x >= 3)], [1, 2, 3])),
     ["campaign"]),
    ("bin_previous", 
     FunctionTransformer(
         func=lambda x: np.select([(x == 0), (x == 1), (x >= 2)], [1, 2, 3])),
     ["previous"]),
    ("pca_econ_vars", pipe_pca, ["emp.var.rate", "nr.employed", "euribor3m"]),
    ("std_idx", StandardScaler(), ["cons.price.idx", "cons.conf.idx"])
])

C:\Users\Silas\Projetos\bank-marketing-classification\.venv\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\Users\Silas\Projetos\bank-marketing-classification\.venv\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [14]:
col_transf.fit_transform(X, y)

C:\Users\Silas\Projetos\bank-marketing-classification\.venv\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\Users\Silas\Projetos\bank-marketing-classification\.venv\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


array([[ 1.        ,  0.13082584,  0.        , ...,  3.24219884,
        -1.49092316,  1.62145457],
       [ 1.        ,  0.11039519,  0.        , ...,  2.02448245,
        -1.18040915, -1.22657471],
       [ 2.        ,  0.07      ,  0.        , ..., -0.97910046,
         0.71890151,  0.88787127],
       ...,
       [ 2.        ,  0.13082584,  0.        , ...,  3.42109286,
        -1.9773951 ,  2.93758931],
       [ 2.        ,  0.11271102,  0.        , ..., -0.97775335,
         0.71890151,  0.88787127],
       [ 1.        ,  0.13082584,  0.        , ..., -1.4149112 ,
         0.5877956 , -0.47141543]])

In [30]:
col_transf.named_transformers_["oh_marital"].get_feature_names_out()

array(['marital_divorced', 'marital_married', 'marital_single'],
      dtype=object)

In [31]:
X

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,29,admin.,married,university.degree,no,no,no,cellular,dec,mon,77,3,999,1,failure,-3.0,92.713,-33.0,0.709,5023.5
1,29,technician,single,university.degree,no,no,no,telephone,may,fri,12,4,999,0,nonexistent,-1.8,92.893,-46.2,1.250,5099.1
2,45,blue-collar,married,basic.6y,unknown,yes,no,telephone,may,thu,277,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0
3,34,services,married,university.degree,no,no,no,cellular,may,thu,70,1,999,1,failure,-1.8,92.893,-46.2,1.327,5099.1
4,32,admin.,single,high.school,no,no,no,cellular,may,fri,1181,9,999,0,nonexistent,-1.8,92.893,-46.2,1.250,5099.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,58,retired,married,professional.course,unknown,no,no,telephone,may,tue,427,2,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
28827,37,management,married,university.degree,no,no,no,telephone,jun,thu,288,1,999,0,nonexistent,1.4,94.465,-41.8,4.961,5228.1
28828,35,admin.,married,high.school,no,yes,no,cellular,oct,thu,194,1,4,1,success,-3.4,92.431,-26.9,0.754,5017.5
28829,40,management,married,university.degree,no,yes,no,telephone,may,wed,295,2,999,0,nonexistent,1.1,93.994,-36.4,4.856,5191.0


In [13]:
te = ce.TargetEncoder()
X_job = te.fit_transform(X["job"], y)

C:\Users\Silas\Projetos\bank-marketing-classification\.venv\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\Users\Silas\Projetos\bank-marketing-classification\.venv\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [14]:
X_job

,job
0,0.130826
1,0.110395
2,0.070000
3,0.083660
4,0.130826
...,...
28826,0.242373
28827,0.112711
28828,0.130826
28829,0.112711
